## IBM Data Science Capstone Project


## Introduction
### Best Nieghborhood for technology Company in Toronto
With over $8 billion in contributions to annual employee wages, Toronto's technology sector is one of the fast-growing in north America. Its great infrastructure, the ease of connection to the rest of the world, good transit, and strong business-centric nature makes Toronto very attractive to international firms and companies. DexTR, a technology company based in San Francisco: engaged in design graphics processing units (GPUs), system chip for smartphone and vehicle navigation processors, is looking to open the second largest center in North America.

After months of regress research and analysis, DexTR has been decided Toronto as the next logical location for expansion in North America. Now that the decision has been made, the objective of this capstone is to analyze and select the optimal location for the DexTR office in the city of Toronto. While not limited to, the criteria below are some of the guiding measures.



## Data Source


### Public safty data portal http://data.torontopolice.on.ca/datasets/98f7dde610b54b9081dfca80be453ac9_0 
### Foursquare API: https://api.foursquare.com/v2/venues/explore
### Walk score: http://api.walkscore.com
### Toronto Nieghborhoods data: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


#### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup # library to parse HTML and XML documents

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries loaded.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

### Scrape Wikipedia page to obtain data and transform into panda datafram using BeatifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #wikipedia link to the table
headers = {'User-Agent': 'Mozilla/5.0'} #setting my browser

In [3]:
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
# creating empty list for the three in the table to store data
postCodeList = []
boroughList = []
neighborList = []

In [5]:
## accessing the table and saving it under vairable
wikitable = soup.find('table', class_="wikitable")

In [6]:
# iterating through the table to get the contents
for row in wikitable.find_all("tr"): #finding row in the table
    cell = row.find_all("td") #finding cells in the row
    if(len(cell) > 0):
        #appending list to corresponding column
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborList.append(cell[2].text.rstrip('\n'))

### Transforming the data in to pandas DataFram

In [7]:
#creating columns for corrosponding lists
columns ={"PostCode": postCodeList, "Borough": boroughList, "Neighborhood": neighborList}
wiki_df = pd.DataFrame(columns)
wiki_df.head(12)

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


## Cleaning data: drop cells with borough not assigned

In [8]:
# drop cells with borough not assigned
wiki_df_dropna = wiki_df[wiki_df.Borough != "Not assigned"].reset_index(drop=True)
wiki_df_dropna.head(10)

,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [9]:
# group neighborhoods with same postal code area
wiki_df_grouped = wiki_df_dropna.groupby(["PostCode", "Borough"],sort=False, as_index=False).agg(lambda x: ", ".join(x))
wiki_df_grouped.reset_index(inplace=True)
wiki_df_grouped.head(10)

,index,PostCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Harbourfront
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,4,M7A,Queen's Park,Not assigned
5,5,M9A,Downtown Toronto,Queen's Park
6,6,M1B,Scarborough,"Rouge, Malvern"
7,7,M3B,North York,Don Mills North
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [10]:
testdf = wiki_df.sort_values("Borough", ascending=False)
testdf.head(10)

,PostCode,Borough,Neighborhood
34,M6C,York,Humewood-Cedarvale
146,M6N,York,Runnymede
145,M6N,York,The Junction North
149,M9N,York,Weston
48,M6E,York,Caledonia-Fairbanks
134,M6M,York,Silverthorn
133,M6M,York,Mount Dennis
132,M6M,York,Keelesdale
131,M6M,York,Del Ray
100,M6K,West Toronto,Brockton


In [11]:
# for a borough with no  neighborhood assigned, set borough as neighborhood 
for index, row in wiki_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
wiki_df_grouped.head(10).set_index("PostCode", inplace=True)
wiki_df_grouped.head(30)

,index,PostCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Harbourfront
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,4,M7A,Queen's Park,Not assigned
5,5,M9A,Downtown Toronto,Queen's Park
6,6,M1B,Scarborough,"Rouge, Malvern"
7,7,M3B,North York,Don Mills North
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Printing the shape of the datafram


In [12]:
wiki_df.shape
wiki_df_grouped.shape

(103, 4)

In [13]:
wiki_df_dropna.groupby(["Borough"]).count()


,PostCode,Neighborhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,44,44
Mississauga,1,1
North York,38,38
Queen's Park,1,1
Scarborough,37,37


In [14]:
#score_url = "http://api.walkscore.com"
!wget -q -O 'score.json' http://api.walkscore.com
print("walk score loaded")

walk score loaded


In [15]:
df_unique = wiki_df_grouped["Neighborhood"].unique()

In [16]:
df_unique

array(['Parkwoods', 'Victoria Village', 'Harbourfront',
       'Lawrence Heights, Lawrence Manor', 'Not assigned', "Queen's Park",
       'Rouge, Malvern', 'Don Mills North',
       'Woodbine Gardens, Parkview Hill', 'Ryerson, Garden District',
       'Glencairn',
       'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park',
       'Highland Creek, Rouge Hill, Port Union',
       'Flemingdon Park, Don Mills South', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Downsview North, Wilson Heights',
       'Thorncliffe Park', 'Adelaide, King, Richmond',
       'Dovercourt Village, Dufferin', 'Scarborough Village',
       'Fairview, Henry Farm, Oriole', 'Northwoo

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
# Load the coordinates from the csv file
coordinates = pd.read_csv(Geospatial_Coordinates)

# renaming postal code column
coordinates.rename(columns={"Postal Code": "PostCode"}, inplace=True)
coordinates.head()


,PostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the table and coordinates

In [19]:
#Merging the table and coordinates
merged_df = wiki_df_grouped.merge(coordinates, on="PostCode", how="left")
merged_df.head(20)

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,7,M3B,North York,Don Mills North,43.745906,-79.352188
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Exploring and clustering the neighborhoods in Toronto using geopy to define a geocoder instance

In [20]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate for city of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate for city of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods with folium

In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='#6a737d',
        fill=True,
        fill_color='#007849',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Using Foursquare API to explore downtown Toronto neighborhood

In [22]:
# The code was removed by Watson Studio for sharing.

## Segment and cluster neighborhoods in downtown Toronto

In [23]:
Downtown_Toronto = merged_df[merged_df["Borough"] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
9,42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [24]:
Downtown_Toronto.head(20)

,index,PostCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
2,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
9,42,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


## Geographical coordinates of downtown Toronto

In [25]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of downtown Toronto are 43.6563221, -79.3809161.


## Creating map for downtown Toronto using the corrosponding latitude and longitude values

In [26]:
# create map of downtown Toronto with latitude & longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

## Using Foursquare API to explore North York neighborhood

In [27]:
Downtown_Toronto.loc[7, 'Neighborhood']

'Adelaide, King, Richmond'

## Get geo coordinate for the above output ('Harbourfront East, Toronto Islands, Union Station', loc7)

In [28]:
# get geo coordinate for Willowdale south
neighborhood_latitude = Downtown_Toronto.loc[7, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Downtown_Toronto.loc[7, 'Longitude'] # neighborhood longitude value

neighborhood_name = Downtown_Toronto.loc[7, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adelaide, King, Richmond are 43.65057120000001, -79.3845675.


## Explore top 100 venues in downtown Toronto with in 999 meters

In [29]:
# Explore top 100 venues in downtown Toronto with in 999 meters
radius = 1999
LIMIT = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ded6c29660a9f4dabf25036'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 240,
  'suggestedBounds': {'ne': {'lat': 43.668562217991024,
    'lng': -79.35974944098126},
   'sw': {'lat': 43.632580182008994, 'lng': -79.40938555901874}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520e5f720e3',
       'name': 'Four Seasons Centre for the Performing Arts',
       'location': {'address': '145 Queen St. W',
        'crossStreet': 'at University Ave.',
        'lat': 43.650592,
        'lng': -79.

## Extracting venue category from Foursquare lab

In [31]:
# function to get venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Transforming json into a pandas dataframe

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Downtown Toronto,Neighborhood,43.653232,-79.385296
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Shangri-La Toronto,Hotel,43.649129,-79.386557
4,Soho House Toronto,Speakeasy,43.648734,-79.386541
5,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
6,The Keg Steakhouse & Bar,Steakhouse,43.649937,-79.384196
7,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,43.650387,-79.388523
8,Pizzeria Libretto,Pizza Place,43.648334,-79.385111
9,JaBistro,Sushi Restaurant,43.649687,-79.388090


## Size of Downtown Toronto dataframe

In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toronto

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Creating dataframe for venues called Downtown_Toronto_venues

In [35]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [36]:
# checking the Downtown_Toronto df shape and printing sample
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(1280, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


## Grouping venues for each nighborhood

In [37]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17
Church and Wellesley,84,84,84,84,84,84
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


## Analyzing Each Neighborhood

In [38]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]

Downtown_Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [39]:
Downtown_Toronto_onehot.shape

(1280, 207)

In [40]:
Downtown_grouped = Downtown_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.00000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.010000,0.0000

## Cluster Neighborhoods in downtown Toronto

In [41]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Salad Place,Bakery,Restaurant,Asian Restaurant,Sushi Restaurant
1,Berczy Park,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
3,"Cabbagetown, St. James Town",Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Bakery,Restaurant,Pub,Pizza Place,Outdoor Sculpture,Sandwich Place
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Japanese Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Bar,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Record Shop
6,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Diner,Baby Store,Convenience Store,Italian Restaurant,Nightclub
7,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Men's Store,Bubble Tea Shop,Mediterranean Restaurant,Gym
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Steakhouse,Gym
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery,American Restaurant


## Create new dataframe cluster each neighborhood

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = Downtown_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head() # check the last columns!

,index,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Chocolate Shop
1,5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Bubble Tea Shop,Italian Restaurant,Tea Room
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,American Restaurant,Beer Bar,Diner,Italian Restaurant,Cosmetics Shop
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant


## Visualize the clusters results

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['PostCode'], Downtown_merged['Borough'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

## Clusters 1

In [46]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Chocolate Shop
2,M5B,-79.378937,0.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Bubble Tea Shop,Italian Restaurant,Tea Room
3,M5C,-79.375418,0.0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,American Restaurant,Beer Bar,Diner,Italian Restaurant,Cosmetics Shop
4,M5E,-79.373306,0.0,Coffee Shop,Steakhouse,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
5,M5G,-79.387383,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Japanese Restaurant
7,M5H,-79.384568,0.0,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Salad Place,Bakery,Restaurant,Asian Restaurant,Sushi Restaurant
8,M5J,-79.381752,0.0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Restaurant,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Park
9,M5K,-79.381576,0.0,Coffee Shop,Café,Hotel,Restaurant,Bar,Seafood Restaurant,Gastropub,Deli / Bodega,Bakery,American Restaurant
10,M5L,-79.379817,0.0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Steakhouse,Gym
12,M5T,-79.400049,0.0,Café,Bar,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Record Shop


## Clusters 2

In [47]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M6G,-79.422564,1.0,Grocery Store,Café,Park,Coffee Shop,Restaurant,Diner,Baby Store,Convenience Store,Italian Restaurant,Nightclub


## Clusters 3

In [48]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M5V,-79.39442,2.0,Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry


## Clusters 4

In [49]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M4W,-79.377529,3.0,Park,Playground,Trail,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Clusters 5

In [50]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,PostCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M5S,-79.400049,4.0,Café,Sandwich Place,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Noodle House,Sushi Restaurant


## Loading Crime data from Public safty data portal to gain some insight

In [51]:

body = client_e2121f57bafa41a3b4483b72a3a1a8a6.get_object(Bucket='courseracapstone-donotdelete-pr-xz5rrczk33f99i',Key='MCI_2014_to_2018.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_crime = pd.read_csv(body)
df_crime.head()


,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,2014,June,20,171,Friday,13,2014.0,June,20.0,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor (76),43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West (36),43.777592,-79.425400,2002
2,-79.425400,43.777592,2351,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1610,100,Robbery With Weapon,2014,July,2,183,Wednesday,2,2014.0,July,2.0,183.0,Wednesday,0,Robbery,D32,36,Newtonbrook West (36),43.777592,-79.425400,2003
3,-79.210373,43.801727,2352,GO-20142412127,2014-07-02T01:30:00.000Z,2014-07-02T05:40:00.000Z,House,2120,200,B&E,2014,July,2,183,Wednesday,5,2014.0,July,2.0,183.0,Wednesday,1,Break and Enter,D42,132,Malvern (132),43.801727,-79.210373,2004
4,-79.254334,43.835884,2354,GO-20142417548,2014-07-02T20:52:00.000Z,2014-07-02T20:57:00.000Z,Commercial,1430,100,Assault,2014,July,2,183,Wednesday,20,2014.0,July,2.0,183.0,Wednesday,20,Assault,D42,130,Milliken (130),43.835884,-79.254334,2005


In [52]:
df_c = df_crime[["MCI", "Neighbourhood", "occurrencedayofyear"]]
crime_df = df_crime.groupby("Neighbourhood").count()
crime_df.tail(10)

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,reportedhour,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Lat,Long,ObjectId
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Willowdale West (37),548,548,548,548,548,548,548,548,548,548,548,548,548,548,548,548,547,547,547,547,547,548,548,548,548,548,548,548
Willowridge-Martingrove-Richview (7),1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081,1081
Woburn (137),3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158,3158
Woodbine Corridor (64),648,648,648,648,648,648,648,648,648,648,648,648,648,648,648,648,646,646,646,646,646,648,648,648,648,648,648,648
Woodbine-Lumsden (60),434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,434,431,431,431,431,431,434,434,434,434,434,434,434
Wychwood (94),623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623,623
Yonge-Eglinton (100),325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325,325
Yonge-St.Clair (97),234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234,234
York University Heights (27),3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141,3141


In [53]:
df_crime.shape

(167525, 29)

## The crime dataframe consists of 167525 crime between 2014 - 2018, I'm going to limit this to just 500 incidents due to computational cost.

In [54]:
limit = 500
df_crime = df_crime.iloc[7:limit, :]

## Toronto's coordinates

In [55]:
latitude = 43.6532
longitude = -79.3832

In [56]:
df_crime.shape

(493, 29)

## Using latitude and longitude to create and display map of Toronto

In [57]:
# create map and display it
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of Toronto
Toronto_map


In [58]:
# instantiate a feature group for the crime dataframe
crime = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_crime.Y, df_crime.X):
    crime.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add incidents to map
Toronto_map.add_child(crime)


In [59]:
# instantiate a feature group for the crime in the dataframe
crime = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_crime.Y, df_crime.X):
    crime.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df_crime.Y)
longitudes = list(df_crime.X)
labels = list(df_crime.MCI)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(Toronto_map)    
    
# add crime to map
Toronto_map.add_child(crime)

In [67]:
from folium import plugins

# let's start again with a clean copy of the map of Toronto
Toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# instantiate a mark cluster object for the incidents in the dataframe
crime = plugins.MarkerCluster().add_to(Toronto_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_crime.Y, df_crime.X, df_crime.MCI):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(crime)

# display map
Toronto_map
